À partir des datasets `population_communes.csv` et `surface_departements.csv`, créer un nouveau dataset qui contient une ligne par département, avec ces colonnes:

- la somme des "Population municipale" du département
- la somme des "Population totale" du département (pour l'explication de la distinction entre "Population municipale" et "Population totale", voir: https://www.insee.fr/fr/metadonnees/definition/c1270)
- la part (en pourcentage) de la population municipale par rapport à la population totale
- la part (en pourcentage) de la population (municipale) du département au sein de sa région
- la densité de la population (municipale) en nb d'habitants / km2

(Le dataset final devrait ressembler à `result-exo-cc.csv`)



In [1]:
dataPath = '../../fspot-infmdi-721/lesson6/'

In [2]:
import pandas as pd

In [3]:
dfPop = pd.read_csv(dataPath + 'population_communes.csv')
dfDept = pd.read_csv(dataPath + 'surface_departements.csv')

In [4]:
dfPop.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [5]:
dfDept.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


In [6]:
# Solution 1: avec GroupBy + Join
#dfConsoDptTot = dfPop.loc[dfPop['variable'] == 'Population totale', ('Code département', 'variable', 'value')].groupby('Code département').sum()
#dfConsoDptCom = dfPop.loc[dfPop['variable'] == 'Population municipale', ('Code département', 'variable', 'value')].groupby('Code département').sum()
#dfConso = dfConsoDptCom.join(dfConsoDptTot, lsuffix='_communale', rsuffix='_totale').join(dfConsoDptReg)

# Solution 2: avec Pivot, mieux car fonctionne avec multi-catégorie
dfConso = dfPop[['Code département', 'variable', 'value']].groupby(['Code département', 'variable']).sum().reset_index()
dfConso = dfConso.pivot('Code département', 'variable', 'value')

In [7]:
dfConso['pct_pop_municipale'] = dfConso['Population municipale'] / dfConso['Population totale'] * 100

In [8]:
dfConso.head() #reset_index()

variable,Population municipale,Population totale,pct_pop_municipale
Code département,,,
01,638425,655171,97.444026
02,536136,549587,97.552526
03,339384,349336,97.151167
04,162565,167331,97.151753
05,141107,146148,96.550757


# Part de la région

In [9]:
dfConsoDptReg = dfPop.loc[:, ('Code département', 'Code région')].groupby('Code département').first()
dfConsoReg = dfPop.loc[dfPop['variable'] == 'Population municipale', ('Code région', 'variable', 'value')].groupby('Code région').sum() \
    .rename(columns={'value':'pop municipale région'})

In [10]:
dfConsoReg.head()

,pop municipale région
Code région,
1,394110
2,376480
3,269352
4,852924
11,12117132


In [11]:
dfConso2 = dfConsoDptReg.reset_index().merge(dfConsoReg.reset_index()).merge(dfConso.reset_index())
dfConso2['part_region']  = dfConso2['Population municipale'] / dfConso2['pop municipale région'] * 100

In [12]:
dfConso2.head()

,Code département,Code région,pop municipale région,Population municipale,Population totale,pct_pop_municipale,part_region
0,01,84,7916889,638425,655171,97.444026,8.064089
1,03,84,7916889,339384,349336,97.151167,4.286835
2,07,84,7916889,325157,334591,97.180438,4.107131
3,15,84,7916889,145969,151615,96.276094,1.843767
4,26,84,7916889,508006,522276,97.267728,6.416738


# Densité

In [13]:
dfConso3 = dfConso2.merge( dfDept, left_on='Code département', right_on='code_insee')
dfConso3['densite'] = dfConso3['Population municipale'] / dfConso3['surf_km2']

In [14]:
dfConso3.head()

,Code département,Code région,pop municipale région,Population municipale,Population totale,pct_pop_municipale,part_region,code_insee,nom,surf_km2,densite
0,01,84,7916889,638425,655171,97.444026,8.064089,01,Ain,5784.0,110.377766
1,03,84,7916889,339384,349336,97.151167,4.286835,03,Allier,7379.0,45.993224
2,07,84,7916889,325157,334591,97.180438,4.107131,07,Ardèche,5566.0,58.418433
3,15,84,7916889,145969,151615,96.276094,1.843767,15,Cantal,5774.0,25.280395
4,26,84,7916889,508006,522276,97.267728,6.416738,26,Drôme,6558.0,77.463556


In [15]:
dfExp = pd.read_csv(dataPath + 'result-exo-cc.csv')
dfExp.head()

,nom_dept,code_dept,code_region,pop_municipale,pop_totale,pct_pop_municipale,part_region,densite
0,Ain,01,84,638425,655171,97.444026,8.064089,110.377766
1,Aisne,02,32,536136,549587,97.552526,8.925380,72.343274
2,Allier,03,84,339384,349336,97.151167,4.286835,45.993224
3,Alpes-de-Haute-Provence,04,93,162565,167331,97.151753,3.237103,23.246818
4,Hautes-Alpes,05,93,141107,146148,96.550757,2.809817,24.768650


In [16]:
dfConso3.drop(['code_insee', 'surf_km2', 'pop municipale région'], inplace=True, axis=1)

In [17]:
dfConso3.rename(columns={'Code département': 'code_dept', 'Population municipale': 'pop_municipale', 
                        'Population totale':'pop_totale', 'Code région': 'code_region',
                        'nom': 'nom_dept'}, inplace=True)

In [18]:
dfFinal = dfConso3.set_index('code_dept').sort_index().reset_index()[[*dfExp.columns]]

In [19]:
dfFinal.head()

,nom_dept,code_dept,code_region,pop_municipale,pop_totale,pct_pop_municipale,part_region,densite
0,Ain,01,84,638425,655171,97.444026,8.064089,110.377766
1,Aisne,02,32,536136,549587,97.552526,8.925380,72.343274
2,Allier,03,84,339384,349336,97.151167,4.286835,45.993224
3,Alpes-de-Haute-Provence,04,93,162565,167331,97.151753,3.237103,23.246818
4,Hautes-Alpes,05,93,141107,146148,96.550757,2.809817,24.768650
